## Extração por máscara - ad solos

In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.features
from shapely.geometry import shape
import warnings

# --- 1. Configuração dos Arquivos ---
VECTOR_FILE = r"resultados_hidrologia_basica\inundacao.geojson"
RASTER_FILE = R"ad_solo\AD_solos.tif"
OUTPUT_FILE = "inundacao_segmentada_por_solo.geojson"

print(f"Lendo vetor: {VECTOR_FILE}")
gdf_inundacao = gpd.read_file(VECTOR_FILE)

# Ignora avisos comuns de CRS que podem não ser relevantes
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- 2. Abrir Raster e Reprojetar Vetor (CRÍTICO PARA PRECISÃO) ---
with rasterio.open(RASTER_FILE) as src_raster:
    raster_crs = src_raster.crs
    print(f"CRS do Raster: {raster_crs}")

    # Garante que o vetor esteja no MESMO CRS do raster
    if gdf_inundacao.crs != raster_crs:
        print(f"Reprojetando vetor de {gdf_inundacao.crs} para {raster_crs}...")
        gdf_inundacao = gdf_inundacao.to_crs(raster_crs)
    else:
        print("Vetor e Raster já estão no mesmo CRS. Ótimo.")

    # --- 3. Mascarar (Clipar) o Raster ---
    # Isso recorta o raster para a exata extensão do seu arquivo geojson
    print("Recortando (mascarando) o raster para a área de interesse...")

    # Extrai as geometrias do GeoDataFrame para usar na máscara
    geoms = [feature["geometry"] for feature in gdf_inundacao.iterfeatures()]

    try:
        out_image, out_transform = rasterio.mask.mask(
            src_raster,
            geoms,
            crop=True,
            filled=True, # Preenche áreas fora da máscara
            nodata=src_raster.nodata # Usa o valor nodata do raster original
        )

        # Pega o valor nodata para filtrar depois
        nodata_value = src_raster.nodata
        # Se não houver nodata definido, escolhemos um improvável (ex: -9999)
        if nodata_value is None:
            nodata_value = -9999
            out_image[out_image == 0] = nodata_value # Assumindo que 0 era nodata

        # Atualiza os metadados para o novo raster recortado
        out_meta = src_raster.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": nodata_value
        })

    except ValueError as e:
        print(f"Erro: {e}. Isso pode acontecer se o vetor não sobrepuser o raster.")
        exit()


# --- 4. Vetorizar o Raster Mascarado (Raster -> Polígono) ---
print("Vetorizando o raster recortado (criando polígonos)...")

# Usamos out_image[0] pois a máscara retorna um array 3D (banda, altura, largura)
# Asseguramos que os dados sejam inteiros para a vetorização de classes
image = out_image[0].astype('int32')

# rasterio.features.shapes gera as formas e os valores dos pixels
# O gerador retorna pares (geometria, valor)
shapes = rasterio.features.shapes(image, transform=out_transform)

polygons = []
values = []
for geom, value in shapes:
    # Filtra os polígonos que são da área "nodata"
    if value != nodata_value:
        polygons.append(shape(geom))
        values.append(value)

# Cria um novo GeoDataFrame com os polígonos do solo
gdf_solos = gpd.GeoDataFrame(
    data={'valor_solo': values},
    geometry=polygons,
    crs=raster_crs
)
print(f"Foram criados {len(gdf_solos)} polígonos de classes de solo.")

# --- 5. Intersectar (Overlay) ---
# Esta é a operação principal que "corta" as linhas
print("Intersectando feições de inundação com os polígonos de solo...")
# 'how='intersection'' garante que só teremos as partes das linhas
# que ESTÃO DENTRO de um polígono de solo.
gdf_intersected = gpd.overlay(
    gdf_inundacao,
    gdf_solos,
    how='intersection',
    keep_geom_type=True
)

# O resultado (gdf_intersected) agora tem as colunas do gdf_inundacao
# E também a coluna 'valor_solo' do gdf_solos.
# As linhas originais foram divididas nas fronteiras dos polígonos.

# --- 6. Salvar Resultado ---
print(f"Salvando resultado em: {OUTPUT_FILE}")
gdf_intersected.to_file(OUTPUT_FILE, driver="GeoJSON")

print("\nProcesso concluído!")
print(f"O arquivo '{OUTPUT_FILE}' contém os segmentos de inundação, cada um com uma coluna 'valor_solo' correspondente.")

Lendo vetor: resultados_hidrologia_basica\inundacao.geojson
CRS do Raster: EPSG:4326
Vetor e Raster já estão no mesmo CRS. Ótimo.
Recortando (mascarando) o raster para a área de interesse...
Vetorizando o raster recortado (criando polígonos)...
Foram criados 142 polígonos de classes de solo.
Intersectando feições de inundação com os polígonos de solo...
Salvando resultado em: inundacao_segmentada_por_solo.geojson

Processo concluído!
O arquivo 'inundacao_segmentada_por_solo.geojson' contém os segmentos de inundação, cada um com uma coluna 'valor_solo' correspondente.


## BUFFER PROPORCIONAL by column

In [2]:
## Parte Buffer proporcional com coluna de referencia (Direto)
import geopandas as gpd
from scipy.optimize import brentq
import os
import math

# --- 1. Configuração de Constantes ---
METRIC_CRS = 'esri:102033'
# Limite de busca em metros (500m é um limite seguro para 10/20/30%)
BRENTQ_UPPER_LIMIT_M = 500.0


# --- 2. Função "Cérebro" (O Otimizador) ---
def find_buffer_distance_for_area(geometry, target_area, search_min, search_max):
    """
    Encontra a distância de buffer necessária para atingir a 'target_area'
    usando scipy.brentq dentro de um range específico.
    """

    # A função que o Scipy tentará zerar:
    def area_difference(distance):
        # Esta é a operação "cara" (demorada)
        return geometry.buffer(distance).area - target_area

    try:
        # Procuramos apenas no range especificado
        distance_needed = brentq(area_difference, a=search_min, b=search_max, xtol=0.01)
        return distance_needed
    except ValueError as e:
        # Re-levanta o erro para ser pego pela função 'calculate_area_buffers'
        raise ValueError(f"Scipy brentq falhou no range [{search_min}, {search_max}m]. {e}")


# --- 3. Função "Fábrica" (O Processamento Principal) ---
def calculate_area_buffers(gdf, metric_crs, percent_change):
    """
    Processa um GeoDataFrame para criar buffers de +/- X% da área original.
    Calcula AMBAS as distâncias (+ e -) para precisão total.
    """
    if gdf.empty:
        return gdf.copy(), gdf.copy()

    # Reprojetar para o CRS métrico para cálculos
    gdf_proj = gdf.to_crs(metric_crs)

    geoms_plus = []
    geoms_minus = []

    factor_plus = 1.0 + (percent_change / 100.0)
    factor_minus = 1.0 - (percent_change / 100.0)

    # Iterar por CADA feição
    for index, row in gdf_proj.iterrows():
        geom = row.geometry
        if geom.is_empty:
            geoms_plus.append(geom)
            geoms_minus.append(geom)
            continue

        area_original_m2 = geom.area
        area_target_plus = area_original_m2 * factor_plus
        area_target_minus = area_original_m2 * factor_minus

        dist_pos = None
        dist_neg = None

        # --- 1. CÁLCULO DO BUFFER POSITIVO (Preciso) ---
        try:
            dist_pos = find_buffer_distance_for_area(
                geom,
                area_target_plus,
                0.0,  # search_min
                BRENTQ_UPPER_LIMIT_M # search_max
            )
        except ValueError:
            # Se falhar, dist_pos permanece None
            pass

        # --- 2. CÁLCULO DO BUFFER NEGATIVO (Preciso) ---
        # Esta é a parte que você pediu para re-adicionar (sem otimização)
        try:
            dist_neg = find_buffer_distance_for_area(
                geom,
                area_target_minus,
                -BRENTQ_UPPER_LIMIT_M, # search_min
                0.0  # search_max
            )
        except ValueError:
            # Se falhar, dist_neg permanece None
            pass

        # --- 3. Aplicar os buffers ---
        # Se o cálculo falhou (dist é None), usa a geometria original
        geoms_plus.append(geom.buffer(dist_pos) if dist_pos is not None else geom)
        geoms_minus.append(geom.buffer(dist_neg) if dist_neg is not None else geom)

    # --- 4. Criar os GeoDataFrames finais ---
    gdf_plus = gdf_proj.copy()
    gdf_plus['geometry'] = geoms_plus

    gdf_minus = gdf_proj.copy()
    gdf_minus['geometry'] = geoms_minus

    return gdf_plus, gdf_minus

In [3]:
# --- PARÂMETROS DE CONFIGURAÇÃO (Versatilidade) ---
goem_path = r'inundacao_segmentada_por_solo.geojson'
output_dir = 'bff_prop_col_direct'
REFERENCE_COLUMN = 'valor_solo' # Nome da coluna que dita a porcentagem
FINAL_MERGE_FILENAME = 'inundacao_adsolo_buffer.geojson'

# --- FIM DOS PARÂMETROS ---

# ==============================================================================
# --- REFINAMENTO COM BASE CIENTÍFICA (Embrapa AD, 2022) ---
#
# Hipótese:
# AD1/AD2 (Baixa AD ~ Arenoso): Alta infiltração -> Buffer Negativo (diminui mancha)
# AD3/AD4 (Média AD ~ Franco): Infiltração moderada -> Buffer Neutro
# AD5/AD6 (Alta AD ~ Argiloso): Baixa infiltração -> Buffer Positivo (aumenta mancha)
#
# Referência: "Classes de água disponível do solo para uso no Zoneamento
# Agrícola de Risco Climático" (Embrapa, 2022)
# ==============================================================================
PERCENTAGE_MAPPING = {
    # Valor 1 (AD1: 0,34 - 0,46): Solo com baixíssima AD (Arenoso)
    1.0: -40.0,

    # Valor 2 (AD2: 0,46 - 0,61): Solo com baixa AD (Arenoso)
    2.0: -20.0,

    # Valor 3 (AD3: 0,61 - 0,80): Solo com média AD (Franco)
    3.0: 0.0,

    # Valor 4 (AD4: 0,80 - 1,06): Solo com média-alta AD (Franco-Argiloso)
    4.0: 5.0, # Um pequeno buffer positivo, pois já tende a saturar

    # Valor 5 (AD5: 1,06 - 1,40): Solo com alta AD (Argiloso)
    5.0: 20.0,

    # Valor 6 (AD6: >= 1,40): Solo com altíssima AD (Argiloso/Saturado)
    6.0: 35.0
}

# (O restante do seu código do Notebook 2 continua aqui...)
# --- FIM DOS PARÂMETROS ---

# --- VARIÁVEIS DE AMBIENTE/CONSTANTES FALTANTES (SIMULAÇÃO) ---
# Substitua com os valores corretos de onde o código foi executado (célula anterior)
try:
    # Tenta usar as variáveis do escopo global se existirem
    _ = calculate_area_buffers
    _ = METRIC_CRS
except NameError:
    # Se não existirem, simula/define para que o código possa ser executado
    print("AVISO: Usando definições placeholder para calculate_area_buffers e METRIC_CRS. Se estas não estiverem corretas no seu ambiente, o código não funcionará.")
    METRIC_CRS = 'EPSG:32723' # Exemplo para UTM 23S
    def calculate_area_buffers(gdf, crs_metric, abs_percent):
        """Função placeholder para simular a criação dos buffers plus e minus."""
        # Se for um placeholder, apenas retorna o GeoDataFrame original com o CRS alterado
        gdf_proj = gdf.to_crs(crs_metric)
        gdf_plus = gdf_proj.copy()
        gdf_minus = gdf_proj.copy()
        return gdf_plus, gdf_minus
# -------------------------------------------------------------------


def main_direct_buffer(gjson_path, ref_col_name, mapping, out_dir, merge_filename):
    print("--- Início do Processamento de Buffer Proporcional Direto ---")
    print(f"Usando coluna de referência: '{ref_col_name}'")

    # Lista para coletar os resultados do buffer de cada valor
    buffered_gdfs = [] # <--- NOVIDADE: Lista para Coletar GDFs

    # --- 1. LER DADOS ---
    try:
        gdf_original = gpd.read_file(gjson_path)
    except FileNotFoundError:
        print(f"ERRO: Arquivo de entrada não encontrado em '{gjson_path}'.")
        return
    except Exception as e:
        print(f"ERRO: Falha ao carregar GeoDataFrame: {e}")
        return

    # 1.1. Verificação da Coluna
    if ref_col_name not in gdf_original.columns:
        print(f"ERRO: A coluna de referência '{ref_col_name}' não foi encontrada no GeoDataFrame.")
        return

    # Guardar o CRS original para salvar corretamente no final
    original_crs = gdf_original.crs if gdf_original.crs else 'EPSG:4326'

    # --- 2. CONFIGURAR O DIRETÓRIO DE SAÍDA ---
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        print(f"Diretório de saída criado: '{out_dir}'")

    # --- 3. PROCESSAR POR VALOR DA COLUNA ---
    # Encontra todos os valores da coluna que têm um mapeamento
    values_to_process = [
        val for val in gdf_original[ref_col_name].unique() if val in mapping
    ]

    if not values_to_process:
        print("AVISO: Nenhum valor na coluna de referência corresponde ao Mapeamento de Porcentagem.")
        return

    for col_value in sorted(values_to_process):
        percent = mapping[col_value]
        print(f"\n-> Processando valor '{col_value}' com Buffer de {percent:+}%...")

        # Filtrar o GeoDataFrame para o valor atual
        gdf_filtered = gdf_original[gdf_original[ref_col_name] == col_value].copy()

        # A função 'calculate_area_buffers' calcula o buffer 'plus' e 'minus'
        # usando o valor ABSOLUTO da porcentagem.
        gdf_plus_proj, gdf_minus_proj = calculate_area_buffers(
            gdf_filtered,
            METRIC_CRS, # Constante da primeira célula
            abs(percent) # Passa o valor absoluto
        )

        # Seleciona o GeoDataFrame que corresponde ao sinal da porcentagem:
        # Se percent >= 0 (positivo), usa o buffer de aumento de área (plus).
        # Se percent < 0 (negativo), usa o buffer de redução de área (minus).
        is_positive = percent >= 0
        gdf_final_proj = gdf_plus_proj if is_positive else gdf_minus_proj

        # Converte de volta para o CRS original
        gdf_final_crs = gdf_final_proj.to_crs(original_crs)

        # --- ADICIONAR AO LISTA PARA MERGE ---
        buffered_gdfs.append(gdf_final_crs) # <--- NOVIDADE: Coleta o GDF

        # --- 4. SALVAR RESULTADOS INDIVIDUAIS (Opcional, mas mantido) ---
        sign = "mais" if is_positive else "menos"
        abs_percent = int(abs(percent))

        output_path_individual = os.path.join(
            out_dir,
            f'inundacao_ref{col_value}_buffered_{sign}_{abs_percent}pct_preciso.geojson'
        )

        gdf_final_crs.to_file(output_path_individual, driver='GeoJSON')
        print(f"Salvo individualmente: {output_path_individual}")

    # --- 5. MERGE E SALVAMENTO DO ARQUIVO FINAL ---
    if buffered_gdfs:
        print("\n--- Iniciando Merge dos Resultados ---")
        # Concatena todos os GeoDataFrames da lista
        gdf_merged = gpd.GeoDataFrame(
            pd.concat(buffered_gdfs, ignore_index=True),
            crs=original_crs
        ) # Assume que pd.concat está disponível (import pandas as pd)

        # Nome e caminho do arquivo de merge final
        final_merge_path = os.path.join(out_dir, merge_filename)

        # Salva o arquivo de merge
        gdf_merged.to_file(final_merge_path, driver='GeoJSON')
        print(f"*** SUCESSO! Arquivo de Merge Final Salvo em: {final_merge_path} ***")
    else:
        print("AVISO: Nenhuma geometria foi processada para o Merge.")


    print("\n--- Processamento de Buffer Proporcional Direto Concluído ---")


if __name__ == "__main__":
    # Importação do Pandas (necessário para a concatenação)
    try:
        import pandas as pd
    except ImportError:
        print("ERRO: A biblioteca 'pandas' é necessária para a função de merge (pd.concat). Por favor, instale-a.")

    try:
        # Assegura que as funções e constantes da célula anterior estão no escopo global
        # A verificação de 'calculate_area_buffers' e 'METRIC_CRS' foi movida para dentro da função ou simulada acima
        main_direct_buffer(
            gjson_path=goem_path,
            ref_col_name=REFERENCE_COLUMN,
            mapping=PERCENTAGE_MAPPING,
            out_dir=output_dir,
            merge_filename=FINAL_MERGE_FILENAME # Novo argumento
        )
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

--- Início do Processamento de Buffer Proporcional Direto ---
Usando coluna de referência: 'valor_solo'

-> Processando valor '2.0' com Buffer de +25.0%...
Salvo individualmente: bff_prop_col_direct\inundacao_ref2.0_buffered_mais_25pct_preciso.geojson

-> Processando valor '4.0' com Buffer de -15.0%...
Salvo individualmente: bff_prop_col_direct\inundacao_ref4.0_buffered_menos_15pct_preciso.geojson

-> Processando valor '5.0' com Buffer de -25.0%...
Salvo individualmente: bff_prop_col_direct\inundacao_ref5.0_buffered_menos_25pct_preciso.geojson

--- Iniciando Merge dos Resultados ---
*** SUCESSO! Arquivo de Merge Final Salvo em: bff_prop_col_direct\inundacao_adsolo_buffer.geojson ***

--- Processamento de Buffer Proporcional Direto Concluído ---
